In [2]:
import numpy as np
import pandas as pd

%matplotlib inline
from matplotlib import pyplot as plt

In [3]:
movies = pd.read_csv("data/movies.csv")
ratings = pd.read_csv("data/ratings.csv")
links = pd.read_csv("data/links.csv")
tags = pd.read_csv("data/tags.csv")




EDA

In [15]:
movies.shape

(9742, 3)

In [11]:
movies.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


Check types:

In [12]:
movies.dtypes

movieId     int64
title      object
genres     object
dtype: object

No data gaps:

In [13]:
movies.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [138]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
movieId    9742 non-null int64
title      9742 non-null object
genres     9742 non-null object
dtypes: int64(1), object(2)
memory usage: 228.4+ KB


Let's count genres number:

In [144]:
from nltk import regexp_tokenize

def extract_genres(i=-1):
    if i != -1:
        l = []
        g = movies.iloc[i]['genres']
        for token in regexp_tokenize(g, pattern=r"[|]", gaps=True):
            l.append(token)
        return l
    
    s = set()
    for i in range(movies.shape[0]):
        g = movies.iloc[i]['genres']
        for token in regexp_tokenize(g, pattern=r"[|]", gaps=True):
            s.add(token)
    return [g for g in s]

In [145]:
genresList = extract_genres()
print(str(len(genresList)) + " genres total:")
for genre in genresList: print(genre)

20 genres total:
Western
Mystery
Sci-Fi
Musical
IMAX
Animation
Drama
Film-Noir
Thriller
Fantasy
Crime
War
Action
Documentary
Romance
Comedy
(no genres listed)
Children
Adventure
Horror


How it clearly can be seen there are some movies without any genres 
specified. Let's count number of such a movies:

In [154]:
genresList.remove("(no genres listed)")
counter = 0
for i in range(movies.shape[0]):
    if movies.iloc[i]['genres'] == "(no genres listed)": counter += 1
print ("Total " + str(counter) + " of " + str(movies.shape[0]) + " " +
    str(counter / movies.shape[0]) + "%" + " movies without any genre specified")

Total 34 of 9742 0.0034900431122972695% movies without any genre specified


It's very uncomfortable to use information about genres in the form 
in which it is for analysis. So, let's create another data frame and 
move all this information there. 

In [157]:
genres = pd.DataFrame()
l = []
for i in range(movies.shape[0]):
    l1 = [i + 1]
    for g in genresList:
        
        if g in extract_genres(i): l1.append(1)
        else: l1.append(0)
    l.append(l1)
    
genres=pd.DataFrame(l, columns=["movieId"] + genresList)

In [161]:
genres.head(5)

,movieId,Western,Mystery,Sci-Fi,Musical,IMAX,Animation,Drama,Film-Noir,Thriller,Fantasy,Crime,War,Action,Documentary,Romance,Comedy,Children,Adventure,Horror
0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,1,1,0
1,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0
3,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0
4,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
